In [1]:
import random
import pandas as pd
import numpy as np
import os
import librosa

from tqdm.auto import tqdm

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

from transformers import Wav2Vec2FeatureExtractor, HubertModel
from glob import glob
from functions import *

from audiomentations import SpecCompose,  SpecChannelShuffle
import torchaudio

import timm
from einops import rearrange
import gc
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
main_path = "D:/meta2/DACON/Corona"
train_df = pd.read_csv('./train_data.csv')
test_df = pd.read_csv('./test_data.csv')
train_id = train_df["id"]
test_id = test_df["id"]

In [3]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(train_df['gender'].values.reshape(-1,1))
train_df = onehot_encoding(ohe, train_df)

test_df = onehot_encoding(ohe, test_df)
test_x = test_df.drop(columns=['id'])

In [4]:
train_x = train_df.drop(columns=['id', 'covid19'])
train_y = train_df['covid19']

In [5]:
train_ages = train_x.age.to_list()
train_ages = list(set(sorted(train_ages)))

In [6]:
#pd to np
train_x = train_x.values
train_y = train_y.values
test_x = test_x.values

In [7]:
age = train_x[:, 0]
mm = MinMaxScaler()
age = mm.fit_transform(age.reshape(-1,1))
age = age.reshape(-1)
train_x[:, 0] = age

In [8]:
age = test_x[:, 0]
age = mm.transform(age.reshape(-1,1))
age = age.reshape(-1)
test_x[:, 0] = age

In [9]:
mm_ages = sorted(list(set(train_x[:, 0])))
ages = {t:m for t, m in zip(train_ages, mm_ages)}
mages = {m:t for t, m in zip(train_ages, mm_ages)}

In [10]:
trainset0, trainset1, trainset2, trainset3, trainset4, trainset5, trainset6 = make_melspectrogram2(main_path, train_id, "train")

  0%|          | 0/3805 [00:00<?, ?it/s]

In [39]:
#unlabeled-data
unlabeled = pd.read_csv('./unlabeled_data.csv')
unlabeled_id = unlabeled["id"]
unlabeled = onehot_encoding(ohe, unlabeled)
unlabeled["covid19"] = 0
unlabeled_y = unlabeled["covid19"]
unlabeled_x = unlabeled.drop(columns = ['id', 'covid19'])

unlabeled_x = unlabeled_x.values
unlabeled_y = unlabeled_y.values

age = unlabeled_x[:, 0]
age = mm.transform(age.reshape(-1,1))
age = age.reshape(-1)
unlabeled_x[:, 0] = age

In [21]:
augment = SpecCompose([SpecChannelShuffle(p=0.5),])

In [22]:
from torch.distributions import Beta
class Mixup(nn.Module):
    def __init__(self, mix_beta):

        super(Mixup, self).__init__()
        self.beta_distribution = Beta(mix_beta, mix_beta)

    def forward(self, X, Y, weight=None):

        bs = X.shape[0]
        n_dims = len(X.shape)
        perm = torch.randperm(bs)
        coeffs = self.beta_distribution.rsample(torch.Size((bs,))).to(X.device)

        if n_dims == 2:
            X = coeffs.view(-1, 1) * X + (1 - coeffs.view(-1, 1)) * X[perm]
        elif n_dims == 3:
            X = coeffs.view(-1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1)) * X[perm]
        else:
            X = coeffs.view(-1, 1, 1, 1) * X + (1 - coeffs.view(-1, 1, 1, 1)) * X[perm]

        Y = coeffs.view(-1, 1) * Y + (1 - coeffs.view(-1, 1)) * Y[perm]

        if weight is None:
            return X, Y
        else:
            weight = coeffs.view(-1) * weight + (1 - coeffs.view(-1)) * weight[perm]
            return X, Y, weight

In [49]:
# PyTorch Dataset
import random
class Pytorch_Dataset(data.Dataset):
    def __init__(self, train_x, train_mels, train_y, transforms, data_type):
        self.x = train_x
        self.y = train_y
        self.transforms = transforms
        self.wav2img = librosa.amplitude_to_db
        self.mels = train_mels
        self.data_type = data_type

    def __getitem__(self, idx):
        inputs1 = self.x[idx]
        inputs2 = self.mels[idx]
        if self.data_type == "train":
            labels = self.y[idx]
            inputs2 = self.transforms(inputs2)
            inputs2 = rearrange(inputs2, 't f c -> c t f')
            inputs2 = self.wav2img(inputs2, ref=np.max)
            return {"inputs1":inputs1, "inputs2":inputs2, "labels":labels}
        elif self.data_type == "valid":
            labels = self.y[idx]
            inputs2 = rearrange(inputs2, 't f c -> c t f')
            inputs2 = self.wav2img(inputs2, ref=np.max)
            return {"inputs1":inputs1, "inputs2":inputs2, "labels":labels}
        elif self.data_type == "test":
            inputs2 = rearrange(inputs2, 't f c -> c t f')
            inputs2 = self.wav2img(inputs2, ref=np.max)
            return {"inputs1":inputs1, "inputs2":inputs2}
    def __len__(self):
        return len(self.x)

In [52]:
class CNN(nn.Module):
    def __init__(self, features=1792+6, net = 64):
        super().__init__()
        self.mixup = Mixup(mix_beta=1)
        self.model = timm.create_model("efficientnetv2_rw_s", pretrained=True, \
                                       num_classes=1)
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.fc0 = nn.Linear(features, net)
        self.fc1 = nn.Linear(net, 1)
    def forward(self, inputs, mode):
        if mode == "train":
            x0 = inputs["inputs1"]
            x1 = inputs["inputs2"]
            y = inputs["labels"]
            #
            y = y.view(-1,1)
            b, c, t, f = x1.shape
            x1 = x1.permute(0, 2, 1, 3)
            #           
            x1, y = self.mixup(x1, y)
            #
            x1 = x1.reshape(b, t, c, f)
            x1 = x1.permute(0, 2, 1, 3)
            #
            x1 = self.model.forward_features(x1)
            x1 = self.avg(x1)
            x1 = x1.flatten(start_dim=1, end_dim=-1)
            x = torch.cat([x0, x1], dim=1)
            x = self.fc0(x)
            x = self.fc1(x)
            return x.view(-1), y.view(-1)
        else:
            x0 = inputs["inputs1"]
            x1 = inputs["inputs2"]
            x1 = self.model.forward_features(x1)
            x1 = self.avg(x1)
            x1 = x1.flatten(start_dim=1, end_dim=-1)
            x = torch.cat([x0, x1], dim=1)
            x = self.fc0(x)
            x = self.fc1(x)
            return x.view(-1)

In [27]:
def train(loader, model, criterion, optimizer, epoch, cuda, log_interval, max_norm=1, verbose=True):
    model.train()
    global_epoch_loss = 0
    samples = 0
    for batch_idx, data_dict in enumerate(loader):
        if cuda:
            data_dict["inputs1"] = data_dict["inputs1"].float().cuda()
            data_dict["inputs2"] = data_dict["inputs2"].float().cuda()
            data_dict["labels"] = data_dict["labels"].float().cuda()
        optimizer.zero_grad()
        output, target = model(data_dict, "train")
        loss = criterion(output, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm)
        optimizer.step()
        global_epoch_loss += loss.data.item() * len(target)
        samples += len(target)
        if verbose and (batch_idx % log_interval == 0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, samples, len(loader.dataset), 100*samples/len(loader.dataset), global_epoch_loss/samples))
    return global_epoch_loss / samples

In [43]:
def valid(loader, model, criterion, epoch, cuda, verbose=True):
    model.eval()
    test_loss = 0
    tpred = []
    ttarget = []

    with torch.no_grad():
        for data_dict in loader:
            if cuda:
                data_dict["inputs1"] = data_dict["inputs1"].float().cuda()
                data_dict["inputs2"] = data_dict["inputs2"].float().cuda()
                data_dict["labels"] = data_dict["labels"].float().cuda()
            output,target = model(data_dict, "train")
            pred = output.sigmoid()
            
            loss = criterion(output, target).data.item()
            test_loss += loss * len(target) # sum up batch loss 
            
            tpred.append(pred.cpu().numpy())
            ttarget.append(target.cpu().numpy())
    test_loss /= len(loader.dataset)
    if verbose:
        print('\n{} Valid: Average loss: {:.4f}'.format(
            epoch, test_loss))
    
#     tpred = np.concatenate(tpred)
#     ttarget = np.concatenate(ttarget)
    
#     best_f1 = 0.
#     best_ths = 0
#     for a in np.arange(0, 0.3, 0.001):
#         tpred = np.where(tpred >= a, 1, 0)
#         tpred = np.array(tpred, dtype=np.int32)
#         ttarget = np.array(ttarget, dtype=np.int32)
#         f1 = f1_score(ttarget, tpred, average="macro")
        
#         if f1 > best_f1:
#             best_f1 = f1
#             best_ths = a
            
    return test_loss

In [44]:
def evaluate(test_loader, model):
    tpred = []
    ttarget = []
    with torch.no_grad():
        for data_dict in test_loader:
            data_dict["inputs1"] = data_dict["inputs1"].float().cuda()
            data_dict["inputs2"] = data_dict["inputs2"].float().cuda()
            output = model(data_dict, "test")
            pred = output.sigmoid()
            tpred.append(pred.cpu().numpy())    
    tpred = np.concatenate(tpred)
    tpred = np.where(tpred >= 0.5, 1, 0)
    tpred = np.array(tpred, dtype=np.int32)
    return tpred

In [105]:
sk = StratifiedKFold(n_splits = 20, shuffle=True, random_state=0)
for folds, (tdx, vdx) in enumerate(sk.split(train_x, train_y)):    
    model = CNN().cuda()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss(reduction='mean')
    log_interval = 100
    cuda = True
    best_loss = 1
    epoch = 1
    x_info_train, x_info_valid = train_x[tdx], train_x[vdx]                  
    y_train, y_valid = train_y[tdx], train_y[vdx]
    Epochs = 10
    for Epoch in range(Epochs):
        for t in range(7):
            if t == 0:
                trainset = trainset0
            elif t == 1:
                trainset = trainset1
            elif t == 2:
                trainset = trainset2
            elif t == 3:
                trainset = trainset3
            elif t == 4:
                trainset = trainset4
            elif t == 5:
                trainset = trainset5
            elif t == 6:
                trainset = trainset6
            
            x_wav_train, x_wav_valid = trainset[tdx], trainset[vdx]

            train_dataset = Pytorch_Dataset(x_info_train, x_wav_train, y_train, augment, "train")
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,\
                                                       num_workers=0)

            valid_dataset = Pytorch_Dataset(x_info_valid, x_wav_valid, y_valid, augment, "train")
            valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16,\
                                                       num_workers=0)

            train_loss = train(train_loader, model, criterion, optimizer, Epoch, cuda,\
                  log_interval, max_norm=1, verbose=True)
            val_loss = valid(valid_loader, model, criterion, Epoch, cuda)
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), main_path + f"/weights7/weight_{folds}_{best_loss}")

            del x_wav_train
            del x_wav_valid
            del train_dataset
            del train_loader
            del valid_dataset
            del valid_loader
            gc.collect()

Train Epoch: 0 [16/3614 (0%)]	Loss: 0.659536
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.299286
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.295941

0 Valid: Average loss: 0.2994
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.473949
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.275168
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.280060

0 Valid: Average loss: 0.3296
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.473189
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.270213
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.284132

0 Valid: Average loss: 0.3089
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.479706
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.262052
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.276993

0 Valid: Average loss: 0.2730
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.426551
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.260489
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.274467

0 Valid: Average loss: 0.2980
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.629885
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.259859
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.280485


6 Valid: Average loss: 0.2962
Train Epoch: 6 [16/3614 (0%)]	Loss: 0.465899
Train Epoch: 6 [1616/3614 (45%)]	Loss: 0.252384
Train Epoch: 6 [3216/3614 (89%)]	Loss: 0.269469

6 Valid: Average loss: 0.2618
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.594623
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.251114
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.273345

7 Valid: Average loss: 0.2768
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.643535
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.251163
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.266738

7 Valid: Average loss: 0.2753
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.572858
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.259352
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.265158

7 Valid: Average loss: 0.2775
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.473505
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.258632
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.273205

7 Valid: Average loss: 0.2655
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.507391
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.268084
Train Epoch: 7 [

Train Epoch: 3 [3216/3614 (89%)]	Loss: 0.272558

3 Valid: Average loss: 0.2818
Train Epoch: 3 [16/3614 (0%)]	Loss: 0.651129
Train Epoch: 3 [1616/3614 (45%)]	Loss: 0.257079
Train Epoch: 3 [3216/3614 (89%)]	Loss: 0.269886

3 Valid: Average loss: 0.2598
Train Epoch: 3 [16/3614 (0%)]	Loss: 0.521467
Train Epoch: 3 [1616/3614 (45%)]	Loss: 0.250986
Train Epoch: 3 [3216/3614 (89%)]	Loss: 0.266628

3 Valid: Average loss: 0.2889
Train Epoch: 4 [16/3614 (0%)]	Loss: 0.455052
Train Epoch: 4 [1616/3614 (45%)]	Loss: 0.260016
Train Epoch: 4 [3216/3614 (89%)]	Loss: 0.273103

4 Valid: Average loss: 0.2573
Train Epoch: 4 [16/3614 (0%)]	Loss: 0.824531
Train Epoch: 4 [1616/3614 (45%)]	Loss: 0.259040
Train Epoch: 4 [3216/3614 (89%)]	Loss: 0.274579

4 Valid: Average loss: 0.2744
Train Epoch: 4 [16/3614 (0%)]	Loss: 0.574985
Train Epoch: 4 [1616/3614 (45%)]	Loss: 0.260648
Train Epoch: 4 [3216/3614 (89%)]	Loss: 0.267547

4 Valid: Average loss: 0.2825
Train Epoch: 4 [16/3614 (0%)]	Loss: 0.649868
Train Epoch: 4 [

Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.260902
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.273441

0 Valid: Average loss: 0.2922
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.456240
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.261291
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.271072

0 Valid: Average loss: 0.2794
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.522402
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.261581
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.273599

0 Valid: Average loss: 0.2837
Train Epoch: 0 [16/3614 (0%)]	Loss: 0.540360
Train Epoch: 0 [1616/3614 (45%)]	Loss: 0.256461
Train Epoch: 0 [3216/3614 (89%)]	Loss: 0.268067

0 Valid: Average loss: 1.0670
Train Epoch: 1 [16/3614 (0%)]	Loss: 0.532089
Train Epoch: 1 [1616/3614 (45%)]	Loss: 0.269098
Train Epoch: 1 [3216/3614 (89%)]	Loss: 0.274441

1 Valid: Average loss: 0.3566
Train Epoch: 1 [16/3614 (0%)]	Loss: 0.410532
Train Epoch: 1 [1616/3614 (45%)]	Loss: 0.256065
Train Epoch: 1 [3216/3614 (89%)]	Loss: 0.269416

1 Valid: Average loss: 0.2893
Train Epoch: 

Train Epoch: 7 [16/3614 (0%)]	Loss: 0.473873
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.252134
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.269178

7 Valid: Average loss: 0.2666
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.473878
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.263424
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.268994

7 Valid: Average loss: 0.2984
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.465568
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.244522
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.258993

7 Valid: Average loss: 0.3049
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.475471
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.267012
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.271255

7 Valid: Average loss: 0.2770
Train Epoch: 7 [16/3614 (0%)]	Loss: 0.572384
Train Epoch: 7 [1616/3614 (45%)]	Loss: 0.253967
Train Epoch: 7 [3216/3614 (89%)]	Loss: 0.266455

7 Valid: Average loss: 0.2711
Train Epoch: 8 [16/3614 (0%)]	Loss: 0.567933
Train Epoch: 8 [1616/3614 (45%)]	Loss: 0.240147
Train Epoch: 8 [3216/3614 (89%)]	Loss: 0.260615

...


0 Valid: Average loss: 0.2619
Train Epoch: 0 [16/3615 (0%)]	Loss: 0.590352
Train Epoch: 0 [1616/3615 (45%)]	Loss: 0.279532
Train Epoch: 0 [3216/3615 (89%)]	Loss: 0.283885

0 Valid: Average loss: 0.2671
Train Epoch: 0 [16/3615 (0%)]	Loss: 0.523119
Train Epoch: 0 [1616/3615 (45%)]	Loss: 0.263521
Train Epoch: 0 [3216/3615 (89%)]	Loss: 0.272866

0 Valid: Average loss: 0.2886
Train Epoch: 0 [16/3615 (0%)]	Loss: 0.482576
Train Epoch: 0 [1616/3615 (45%)]	Loss: 0.259745
Train Epoch: 0 [3216/3615 (89%)]	Loss: 0.271455

0 Valid: Average loss: 0.2565
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.583430
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.249242
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.266295

1 Valid: Average loss: 0.2954
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.610142
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.275545
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.286364

1 Valid: Average loss: 0.3011
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.703432
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.271785
Train Epoch: 1 [

Train Epoch: 7 [3216/3615 (89%)]	Loss: 0.277432

7 Valid: Average loss: 0.2654
Train Epoch: 7 [16/3615 (0%)]	Loss: 0.456097
Train Epoch: 7 [1616/3615 (45%)]	Loss: 0.247697
Train Epoch: 7 [3216/3615 (89%)]	Loss: 0.267632

7 Valid: Average loss: 0.3237
Train Epoch: 7 [16/3615 (0%)]	Loss: 0.642013
Train Epoch: 7 [1616/3615 (45%)]	Loss: 0.255730
Train Epoch: 7 [3216/3615 (89%)]	Loss: 0.271017

7 Valid: Average loss: 0.2681
Train Epoch: 7 [16/3615 (0%)]	Loss: 0.547261
Train Epoch: 7 [1616/3615 (45%)]	Loss: 0.258487
Train Epoch: 7 [3216/3615 (89%)]	Loss: 0.268640

7 Valid: Average loss: 0.2742
Train Epoch: 7 [16/3615 (0%)]	Loss: 0.518340
Train Epoch: 7 [1616/3615 (45%)]	Loss: 0.250188
Train Epoch: 7 [3216/3615 (89%)]	Loss: 0.268031

7 Valid: Average loss: 0.2892
Train Epoch: 8 [16/3615 (0%)]	Loss: 0.615453
Train Epoch: 8 [1616/3615 (45%)]	Loss: 0.254962
Train Epoch: 8 [3216/3615 (89%)]	Loss: 0.266763

8 Valid: Average loss: 0.3011
Train Epoch: 8 [16/3615 (0%)]	Loss: 0.514066
Train Epoch: 8 [

Train Epoch: 4 [1616/3615 (45%)]	Loss: 0.266479
Train Epoch: 4 [3216/3615 (89%)]	Loss: 0.281792

4 Valid: Average loss: 0.3063
Train Epoch: 4 [16/3615 (0%)]	Loss: 0.429857
Train Epoch: 4 [1616/3615 (45%)]	Loss: 0.269239
Train Epoch: 4 [3216/3615 (89%)]	Loss: 0.274967

4 Valid: Average loss: 0.2513
Train Epoch: 4 [16/3615 (0%)]	Loss: 0.298734
Train Epoch: 4 [1616/3615 (45%)]	Loss: 0.263825
Train Epoch: 4 [3216/3615 (89%)]	Loss: 0.275554

4 Valid: Average loss: 0.2916
Train Epoch: 4 [16/3615 (0%)]	Loss: 0.235078
Train Epoch: 4 [1616/3615 (45%)]	Loss: 0.273687
Train Epoch: 4 [3216/3615 (89%)]	Loss: 0.269859

4 Valid: Average loss: 0.2763
Train Epoch: 4 [16/3615 (0%)]	Loss: 0.408315
Train Epoch: 4 [1616/3615 (45%)]	Loss: 0.266432
Train Epoch: 4 [3216/3615 (89%)]	Loss: 0.265609

4 Valid: Average loss: 0.2508
Train Epoch: 4 [16/3615 (0%)]	Loss: 0.379853
Train Epoch: 4 [1616/3615 (45%)]	Loss: 0.253183
Train Epoch: 4 [3216/3615 (89%)]	Loss: 0.263463

4 Valid: Average loss: 0.2984
Train Epoch: 

Train Epoch: 1 [16/3615 (0%)]	Loss: 0.652919
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.264309
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.278993

1 Valid: Average loss: 0.2548
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.595012
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.252864
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.272329

1 Valid: Average loss: 0.3107
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.430653
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.254089
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.269030

1 Valid: Average loss: 0.2812
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.643477
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.254116
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.269976

1 Valid: Average loss: 0.2780
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.744783
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.258508
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.272570

1 Valid: Average loss: 0.2708
Train Epoch: 1 [16/3615 (0%)]	Loss: 0.546426
Train Epoch: 1 [1616/3615 (45%)]	Loss: 0.261945
Train Epoch: 1 [3216/3615 (89%)]	Loss: 0.272419


7 Valid: Average loss: 0.2914
Train Epoch: 7 [16/3615 (0%)]	Loss: 0.648668
Train Epoch: 7 [1616/3615 (45%)]	Loss: 0.255898
Train Epoch: 7 [3216/3615 (89%)]	Loss: 0.271320

7 Valid: Average loss: 0.2867
Train Epoch: 8 [16/3615 (0%)]	Loss: 0.593770
Train Epoch: 8 [1616/3615 (45%)]	Loss: 0.255477
Train Epoch: 8 [3216/3615 (89%)]	Loss: 0.271742

8 Valid: Average loss: 0.2703
Train Epoch: 8 [16/3615 (0%)]	Loss: 0.622504
Train Epoch: 8 [1616/3615 (45%)]	Loss: 0.262580
Train Epoch: 8 [3216/3615 (89%)]	Loss: 0.279149

8 Valid: Average loss: 0.2853
Train Epoch: 8 [16/3615 (0%)]	Loss: 0.526895
Train Epoch: 8 [1616/3615 (45%)]	Loss: 0.257684
Train Epoch: 8 [3216/3615 (89%)]	Loss: 0.269320

8 Valid: Average loss: 0.3056
Train Epoch: 8 [16/3615 (0%)]	Loss: 0.351876
Train Epoch: 8 [1616/3615 (45%)]	Loss: 0.265338
Train Epoch: 8 [3216/3615 (89%)]	Loss: 0.277024

8 Valid: Average loss: 0.2769
Train Epoch: 8 [16/3615 (0%)]	Loss: 0.702840
Train Epoch: 8 [1616/3615 (45%)]	Loss: 0.258268
Train Epoch: 8 [

In [54]:
sk = StratifiedKFold(n_splits = 10, shuffle=True, random_state=0)
ths = {}
for folds, (tdx, vdx) in enumerate(sk.split(train_x, train_y)):
    print(folds)
    x_info_train, x_info_valid = train_x[tdx], train_x[vdx]                  
    y_train, y_valid = train_y[tdx], train_y[vdx]
    x_wav_train0, x_wav_valid0 = trainset0[tdx], trainset0[vdx]

    train_dataset = Pytorch_Dataset(x_info_train, x_wav_train0, y_train, augment, "train")
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,\
                                               num_workers=0)
    valid_dataset = Pytorch_Dataset(x_info_valid, x_wav_valid0, y_valid, augment, "valid")
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16,\
                                               num_workers=0)
    
    model = CNN().cuda()
    if folds == 0:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_0_0.22946468848255988"))
    elif folds == 1:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_1_0.2500715496658966"))
    elif folds == 2:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_2_0.24418334596426156"))
    elif folds == 3:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_3_0.2503902230049994"))
    elif folds == 4:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_4_0.24664222052091062"))
    elif folds == 5:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_5_0.24372161477804183"))
    elif folds == 6:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_6_0.23513759829496084"))
    elif folds == 7:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_7_0.2564500821264167"))
    elif folds == 8:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_8_0.2378425337766346"))
    elif folds == 9:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_9_0.24639313001381724"))
    
    
    model.eval()
    tpred = []
    ttarget = []
    
    with torch.no_grad():
        for data_dict in valid_loader:
            if cuda:
                data_dict["inputs1"] = data_dict["inputs1"].float().cuda()
                data_dict["inputs2"] = data_dict["inputs2"].float().cuda()
                data_dict["labels"] = data_dict["labels"].float().cuda()
            target = data_dict["labels"]
            output = model(data_dict, "valid")
            pred = output.sigmoid()
            tpred.append(pred.cpu().numpy())
            ttarget.append(target.cpu().numpy())
            
    tpred = np.concatenate(tpred)
    ttarget = np.concatenate(ttarget)
    
    best_f1 = 0.
    for a in np.arange(0, 0.5, 0.001):
        tpred1 = np.where(tpred >= a, 1, 0)
        tpred1 = np.array(tpred1, dtype=np.int32)
        ttarget1 = np.array(ttarget, dtype=np.int32)
        f1 = f1_score(ttarget1, tpred1, average="macro")
        
        if f1 > best_f1:
            best_f1 = f1
            ths[folds] = [a, best_f1]

0
1
2
3
4
5
6
7
8
9


In [108]:
ths

{0: [0.20600000000000002, 0.6517322219327949],
 1: [0.137, 0.6664828600312472],
 2: [0.107, 0.6448613928329953],
 3: [0.20700000000000002, 0.6443202979515829],
 4: [0.107, 0.6857878549459981],
 5: [0.261, 0.6500920810313076],
 6: [0.213, 0.5777777777777777],
 7: [0.146, 0.7331460674157304],
 8: [0.11900000000000001, 0.6042697214267118],
 9: [0.139, 0.6697088222511951],
 10: [0.135, 0.5984784446322908],
 11: [0.152, 0.5751552795031056],
 12: [0.186, 0.6697088222511951],
 13: [0.135, 0.7216901408450704],
 14: [0.10300000000000001, 0.5661494011836525],
 15: [0.23900000000000002, 0.5432692307692307],
 16: [0.13, 0.6515588158239455],
 17: [0.161, 0.5758928571428571],
 18: [0.138, 0.5960809686662354],
 19: [0.132, 0.6915584415584415]}

In [109]:
ths_mean = []
for i in ths.values():
    ths_mean.append(i[0])
ths_mean = np.mean(ths_mean)
ths_mean

0.15765

In [57]:
testset0, testset1, testset2 = make_melspectrogram1(main_path, test_id, "test")

  0%|          | 0/5732 [00:00<?, ?it/s]

In [60]:
test_dataset = Pytorch_Dataset(test_x, testset0, None, None, "test")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,num_workers=0)

voting = []
prob = []
for folds in range(10):
    model = CNN().cuda()
    if folds == 0:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_0_0.22946468848255988"))
    elif folds == 1:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_1_0.2500715496658966"))
    elif folds == 2:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_2_0.24418334596426156"))
    elif folds == 3:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_3_0.2503902230049994"))
    elif folds == 4:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_4_0.24664222052091062"))
    elif folds == 5:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_5_0.24372161477804183"))
    elif folds == 6:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_6_0.23513759829496084"))
    elif folds == 7:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_7_0.2564500821264167"))
    elif folds == 8:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_8_0.2378425337766346"))
    elif folds == 9:
        model.load_state_dict(torch.load("D:/meta2/DACON/Corona/weights6/weight_9_0.24639313001381724"))
    
    
    model.eval()

    tpred = []
    with torch.no_grad():
        for data_dict in tqdm(test_loader, total = len(test_loader.dataset)):
            data_dict["inputs1"] = data_dict["inputs1"].float().cuda()
            data_dict["inputs2"] = data_dict["inputs2"].float().cuda()
            output = model(data_dict, "test")
            pred = output.sigmoid()
            tpred.append(pred.cpu().numpy())    
    tpred = np.concatenate(tpred)
    a = ths[folds][0]
    tpred1 = np.where(tpred >= a, 1, 0)
    tpred1 = np.array(tpred1, dtype=np.int32)
    voting.append(tpred1)
    prob.append(tpred)

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

  0%|          | 0/5732 [00:00<?, ?it/s]

In [111]:
prob_mean = np.mean(prob, axis = 0)

In [112]:
soft = np.where(prob_mean >= ths_mean, 1, 0)
soft = np.array(soft, dtype=np.int32)

In [113]:
np.save('./probs/effv2s_soft_prob20.npy', arr=prob_mean)

In [114]:
submission = pd.read_csv('./sample_submission.csv')
submission['covid19'] = soft
submission.to_csv('./submit_files/submit24.csv', index=False)